# Spatially Enabled DataFrames

This notebook will working with Spatially Enabled Dataframes and also building a spatial index on dataframe geometries to allow for quick searching, accessing, and processing. The notebook will also demonstrate spatial joins to geoenable dataframes.
In this case we will work with official database of MINSA about the positive cases to Covid19 in Peru

## Importing libraries

In [1]:
import os
import arcpy
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from io import BytesIO
from zipfile import ZipFile
import requests
def csv_from_zip(url):
    request = requests.get(url)
    zf = ZipFile(BytesIO(request.content))
    # find the first matching csv file in the zip:
    match = [s for s in zf.namelist() if ".csv" in s]
    df = pd.read_csv(zf.open(match[0]),encoding="ANSI")
    return df

In [3]:
url = "https://www.datosabiertos.gob.pe/sites/default/files/DATOSABIERTOS_SISCOVID.zip"
positive_cases = csv_from_zip(url)
positive_cases

,UUID,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO
0,7320cabdc1aaca6c59014cae76a134e6,AMAZONAS,BAGUA,BAGUA,PCR,15.0,MASCULINO,7/04/2020
1,e81602051997ace8340bb8c18fe24c65,AMAZONAS,BAGUA,COPALLIN,PCR,73.0,MASCULINO,7/04/2020
2,cecdbf10074dbc011ae05b3cbd320a6f,AMAZONAS,BONGARA,JAZAN,PCR,85.0,MASCULINO,9/04/2020
3,71ecb6bccb248b0bb2ac72ed51b5e979,AMAZONAS,CHACHAPOYAS,HUANCAS,PCR,31.0,MASCULINO,9/04/2020
4,566af4276cbe9359abe93f9aa86396c3,AMAZONAS,BAGUA,COPALLIN,PCR,41.0,MASCULINO,9/04/2020
...,...,...,...,...,...,...,...,...
108764,099e9eeeb47ec70204a5e55c9f5306cb,UCAYALI,CORONEL PORTILLO,CALLERIA,PR,22.0,MASCULINO,22/04/2020
108765,3106814086998cee9282e68aaf4b8d08,UCAYALI,CORONEL PORTILLO,CALLERIA,PR,40.0,MASCULINO,2/05/2020
108766,63b74ab867a3ff1ff2f6068af0fb70a2,UCAYALI,CORONEL PORTILLO,CALLERIA,PR,23.0,MASCULINO,30/04/2020
108767,b964a37db73e3f1fe350b4acf7ce29fc,UCAYALI,CORONEL PORTILLO,YARINACOCHA,PR,64.0,MASCULINO,9/05/2020


## Geoenable the data


In [4]:
from arcpy import env
import arcgis
workspace = r"C:\Users\jf_ph\OneDrive\Documentos\ArcGIS\Projects\Covid19\Covid19.gdb"
outworkspace = r"C:\Users\jf_ph\OneDrive\Documentos\ArcGIS\Projects\Covid19\Outcomes.gdb"
## Define Output Coordinate System 
env.outputCoordinateSystem = arcpy.SpatialReference("WGS 1984")
inFeature = os.path.join(workspace, "DISTRITOS")

#Accesing to district table
district = pd.DataFrame.spatial.from_featureclass(inFeature)[['PROVINCIA', 'DISTRITO','SHAPE']]
district.head()

,PROVINCIA,DISTRITO,SHAPE
0,PUERTO INCA,CODO DEL POZUZO,"{'rings': [[[-75.31797010613548, -9.2952886894..."
1,PUERTO INCA,TOURNAVISTA,"{'rings': [[[-74.64135623450261, -8.8230228515..."
2,PADRE ABAD,ALEXANDER VON HUMBOLDT,"{'rings': [[[-75.02252891407807, -8.7419282648..."
3,PADRE ABAD,IRAZOLA,"{'rings': [[[-75.13863842922007, -8.5671157217..."
4,PADRE ABAD,NESHUYA,"{'rings': [[[-75.01588753947834, -8.4463700152..."


## Cleaning Data

In [128]:
import warnings
warnings.filterwarnings("ignore")

In [39]:
pc = sorted(list(set(positive_cases["PROVINCIA"] + " " + positive_cases["DISTRITO"])))
dp = sorted(list(set(district["PROVINCIA"] + " " + district["DISTRITO"])))
equal = []
notequal = []
for item in pc:
    if item in dp:
        equal.append(item)
    else:
        notequal.append(item)        
notequal         

In [75]:
district[district["PROVINCIA"]  == "NASCA"]

,PROVINCIA,DISTRITO,SHAPE
216,NASCA,NASCA,"{'rings': [[[-74.84531385904734, -14.709147733..."
289,NASCA,VISTA ALEGRE,"{'rings': [[[-74.69321885896312, -14.838610982..."
295,NASCA,CHANGUILLO,"{'rings': [[[-75.36096286024775, -14.599394482..."
854,NASCA,MARCONA,"{'rings': [[[-74.69058598395594, -14.999142608..."
907,NASCA,EL INGENIO,"{'rings': [[[-74.8895197338486, -14.5474053565..."


In [88]:
positive_cases["PROVINCIA"].loc[positive_cases["PROVINCIA"]  == "NAZCA"] = "NASCA" 
positive_cases["DISTRITO"].loc[positive_cases["DISTRITO"]  == "NAZCA"] = "NASCA"

15364    NASCA
15366    NASCA
15382    NASCA
15383    NASCA
15384    NASCA
         ...  
17204    NASCA
17205    NASCA
17206    NASCA
17207    NASCA
17211    NASCA
Name: PROVINCIA, Length: 120, dtype: object

In [106]:
district.loc[district["PROVINCIA"]  == "HUAMANGA"]

,PROVINCIA,DISTRITO,SHAPE
1815,HUAMANGA,OCROS,"{'rings': [[[-73.84650607879246, -13.259040937..."
1816,HUAMANGA,CHIARA,"{'rings': [[[-74.19223383072665, -13.205974622..."
1818,HUAMANGA,CARMEN ALTO,"{'rings': [[[-74.22411910860995, -13.170050730..."
1819,HUAMANGA,SAN JUAN BAUTISTA,"{'rings': [[[-74.20557028024422, -13.169823179..."
1820,HUAMANGA,ACOCRO,"{'rings': [[[-74.08726003772051, -13.164000019..."
1821,HUAMANGA,VINCHOS,"{'rings': [[[-74.35716001242457, -13.168340018..."
1822,HUAMANGA,ANDRES AVELINO CACERES DORREGARAY,"{'rings': [[[-74.19278003936728, -13.136220019..."
1823,HUAMANGA,SOCOS,"{'rings': [[[-74.28186235766162, -13.133139645..."
1824,HUAMANGA,TAMBILLO,"{'rings': [[[-74.18915014896015, -13.115450128..."
1825,HUAMANGA,JESUS NAZARENO,"{'rings': [[[-74.20854003769256, -13.098100019..."


In [ ]:
positive_cases["DISTRITO"].loc[positive_cases["DISTRITO"]  == "ANDRES AVELINO CACERES D."] ="ANDRES AVELINO CACERES DORREGARAY"

In [112]:
positive_cases.loc[positive_cases["DISTRITO"]  == "ANDRES AVELINO CACERES DORREGARAY"]

,UUID,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO
5188,772b4adf4bb4ff6fe00b5a6c65c875e0,AYACUCHO,HUAMANGA,ANDRES AVELINO CACERES DORREGARAY,PCR,35.0,MASCULINO,5/05/2020
5191,13084d88e21db12c3b20c8cca9b5b57d,AYACUCHO,HUAMANGA,ANDRES AVELINO CACERES DORREGARAY,PCR,27.0,MASCULINO,5/05/2020
5192,144a55cc547b7df9ad545255d29ab174,AYACUCHO,HUAMANGA,ANDRES AVELINO CACERES DORREGARAY,PCR,51.0,FEMENINO,5/05/2020
5193,564c964e6b93412d48e44dfd6d9f88e8,AYACUCHO,HUAMANGA,ANDRES AVELINO CACERES DORREGARAY,PCR,16.0,MASCULINO,5/05/2020
5227,771d7184caa30d813f8852b628472f78,AYACUCHO,HUAMANGA,ANDRES AVELINO CACERES DORREGARAY,PCR,78.0,FEMENINO,8/05/2020
5236,19c8450806a8f2a6d0a72d7a5af178d3,AYACUCHO,HUAMANGA,ANDRES AVELINO CACERES DORREGARAY,PCR,25.0,FEMENINO,8/05/2020
5245,da3f7b437160fbe7336739b22c253830,AYACUCHO,HUAMANGA,ANDRES AVELINO CACERES DORREGARAY,PCR,70.0,FEMENINO,13/05/2020
5246,866069c3e38d401c5873af323836ff23,AYACUCHO,HUAMANGA,ANDRES AVELINO CACERES DORREGARAY,PCR,34.0,MASCULINO,13/05/2020
5260,920b1becc18c1a2143a623e43d574026,AYACUCHO,HUAMANGA,ANDRES AVELINO CACERES DORREGARAY,PR,16.0,FEMENINO,16/05/2020
5266,ad885ae37f99c9ac9a754062d89a92dc,AYACUCHO,HUAMANGA,ANDRES AVELINO CACERES DORREGARAY,PR,38.0,FEMENINO,18/05/2020


In [113]:
district.loc[district["PROVINCIA"]  == "CALLAO"]

,PROVINCIA,DISTRITO,SHAPE
553,CALLAO,LA PUNTA,"{'rings': [[[-77.15883611109763, -12.066453105..."
1118,CALLAO,LA PERLA,"{'rings': [[[-77.10443948579359, -12.070195354..."
1223,CALLAO,BELLAVISTA,"{'rings': [[[-77.10086048712174, -12.063072479..."
1288,CALLAO,CARMEN DE LA LEGUA REYNOSO,"{'rings': [[[-77.08316998610593, -12.039246104..."
1295,CALLAO,CALLAO,"{'rings': [[[-77.1121933613345, -11.9735214788..."
1301,CALLAO,VENTANILLA,"{'rings': [[[-77.13520775083697, -11.817821212..."
1834,CALLAO,MI PERU,"{'rings': [[[-77.11508623933844, -11.841718254..."


In [ ]:
positive_cases["DISTRITO"].loc[positive_cases["DISTRITO"]  == "CARMEN DE LA LEGUA-REYNOSO"] = "CARMEN DE LA LEGUA REYNOSO"

In [115]:
positive_cases.loc[positive_cases["DISTRITO"]  == "CARMEN DE LA LEGUA REYNOSO"]

,UUID,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO
6068,5d7241ebf6e3c959db51ed07cc4c45c6,CALLAO,CALLAO,CARMEN DE LA LEGUA REYNOSO,PCR,62.0,FEMENINO,24/03/2020
6069,6bd87f99c39818c56fed7f98d79107e9,CALLAO,CALLAO,CARMEN DE LA LEGUA REYNOSO,PCR,53.0,MASCULINO,24/03/2020
6075,ef350cd2619b836d963d58adbc25e12a,CALLAO,CALLAO,CARMEN DE LA LEGUA REYNOSO,PCR,32.0,FEMENINO,28/03/2020
6082,3d8ad5faa3a8d692c289d90fcfbbde16,CALLAO,CALLAO,CARMEN DE LA LEGUA REYNOSO,PCR,35.0,MASCULINO,29/03/2020
6101,5a90df58cdfabffc5cc697671d1fc134,CALLAO,CALLAO,CARMEN DE LA LEGUA REYNOSO,PCR,53.0,MASCULINO,1/04/2020
...,...,...,...,...,...,...,...,...
13551,fac613a7a32599f5f0cf01950d7710cc,CALLAO,CALLAO,CARMEN DE LA LEGUA REYNOSO,PR,49.0,MASCULINO,23/04/2020
13580,e0657d8cf8fe4535a72b3904693041c6,CALLAO,CALLAO,CARMEN DE LA LEGUA REYNOSO,PR,75.0,MASCULINO,3/05/2020
13585,96dc9f5a14212a1ef7483fa61718c8fc,CALLAO,CALLAO,CARMEN DE LA LEGUA REYNOSO,PR,79.0,MASCULINO,27/04/2020
13706,801776ae4632df3a57e07c283711d4a3,CALLAO,CALLAO,CARMEN DE LA LEGUA REYNOSO,PR,57.0,MASCULINO,9/05/2020


In [116]:
district.loc[district["PROVINCIA"]  == "SANDIA"]

,PROVINCIA,DISTRITO,SHAPE
445,SANDIA,PATAMBUCO,"{'rings': [[[-69.52079547912803, -14.199857482..."
448,SANDIA,SANDIA,"{'rings': [[[-69.39851972910219, -14.146312982..."
450,SANDIA,SAN JUAN DEL ORO,"{'rings': [[[-69.17931810394731, -14.138139482..."
869,SANDIA,QUIACA,"{'rings': [[[-69.26965072899793, -14.334416981..."
877,SANDIA,CUYOCUYO,"{'rings': [[[-69.47617772921876, -14.418951231..."
911,SANDIA,YANAHUAYA,"{'rings': [[[-69.03798435369492, -14.216396357..."
929,SANDIA,PHARA,"{'rings': [[[-69.55719072936864, -13.916519481..."
962,SANDIA,ALTO INAMBARI,"{'rings': [[[-69.44905585534343, -13.828591731..."
1097,SANDIA,SAN PEDRO DE PUTINA PUNCO,"{'rings': [[[-68.8676366036463, -13.0107486062..."
1126,SANDIA,LIMBANI,"{'rings': [[[-69.61954822896757, -13.366496231..."


In [ ]:
positive_cases["DISTRITO"].loc[positive_cases["DISTRITO"]  == "SAN PEDRO DE PUTINA PUNCU"]="SAN PEDRO DE PUTINA PUNCO"

In [118]:
positive_cases.loc[positive_cases["DISTRITO"]  == "SAN PEDRO DE PUTINA PUNCO"]

,UUID,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO
104416,d7e25f11f606e73c8344134e953773cd,PUNO,SANDIA,SAN PEDRO DE PUTINA PUNCO,PR,37.0,FEMENINO,4/05/2020
104455,c436aaedf3cec6dcbc0a276c91c9627f,PUNO,SANDIA,SAN PEDRO DE PUTINA PUNCO,PR,33.0,MASCULINO,4/05/2020
104457,fe9a4d4ccf0005b4571faed3d3c24c2c,PUNO,SANDIA,SAN PEDRO DE PUTINA PUNCO,PR,65.0,MASCULINO,4/05/2020


In [119]:
district.loc[district["PROVINCIA"]  == "SATIPO"]

,PROVINCIA,DISTRITO,SHAPE
470,SATIPO,MAZAMARI - PANGOA,"{'rings': [[[-74.26984098317479, -11.161837229..."
686,SATIPO,SATIPO,"{'rings': [[[-74.526869733951, -11.00897547903..."
743,SATIPO,RIO TAMBO,"{'rings': [[[-73.67751623308766, -10.789927477..."
1072,SATIPO,RIO NEGRO,"{'rings': [[[-74.58634985895878, -10.976857604..."
1274,SATIPO,LLAYLLA,"{'rings': [[[-74.54761310865803, -11.348321353..."
1284,SATIPO,PAMPA HERMOSA,"{'rings': [[[-74.82314848441496, -11.267893103..."
1285,SATIPO,COVIRIALI,"{'rings': [[[-74.63061935899617, -11.264947103..."
1846,SATIPO,VIZCATAN DEL ENE,"{'rings': [[[-74.27124996271556, -12.071320076..."


In [ ]:
positive_cases["DISTRITO"].loc[positive_cases["DISTRITO"]  == "MAZAMARI"] = "MAZAMARI - PANGOA"
positive_cases["DISTRITO"].loc[positive_cases["DISTRITO"]  == "PANGOA"] = "MAZAMARI - PANGOA"

In [125]:
positive_cases.loc[positive_cases["DISTRITO"]  == "MAZAMARI - PANGOA"]

,UUID,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO
17404,0f3dc33ed120f014e83669f1bdc531e6,JUNIN,SATIPO,MAZAMARI - PANGOA,PCR,36.0,MASCULINO,3/05/2020
17499,85a3f6b15e14952b3488228399e762a3,JUNIN,SATIPO,MAZAMARI - PANGOA,PR,23.0,FEMENINO,18/05/2020
17593,902239ffe128ed29ab18fe857990224f,JUNIN,SATIPO,MAZAMARI - PANGOA,PR,41.0,MASCULINO,7/05/2020
17606,6326e44c4a83ae22885e37202fe5c12f,JUNIN,SATIPO,MAZAMARI - PANGOA,PR,21.0,MASCULINO,15/04/2020
17627,e674d9a43578dc7ae33abb0754536b86,JUNIN,SATIPO,MAZAMARI - PANGOA,PR,31.0,MASCULINO,7/05/2020
17629,da5e9964c490a321252e528fcb73dd78,JUNIN,SATIPO,MAZAMARI - PANGOA,PR,33.0,MASCULINO,6/05/2020
17637,1e51353c5ed3d9348386a86717fa538c,JUNIN,SATIPO,MAZAMARI - PANGOA,PR,38.0,MASCULINO,23/04/2020
17638,5408263efcdaea980a181c9911b12789,JUNIN,SATIPO,MAZAMARI - PANGOA,PR,25.0,MASCULINO,6/05/2020
17645,6a21d2408a45a8e11108f729e703b3a2,JUNIN,SATIPO,MAZAMARI - PANGOA,PR,20.0,MASCULINO,15/04/2020
17651,9512bf37b4e04f4821fa066ccc60cda8,JUNIN,SATIPO,MAZAMARI - PANGOA,PR,36.0,MASCULINO,30/04/2020


In [126]:
district.loc[district["PROVINCIA"]  == "TACNA"]

,PROVINCIA,DISTRITO,SHAPE
27,TACNA,CORONEL GREGORIO ALBARRACIN LANCHIPA,"{'rings': [[[-70.23709960456307, -18.022460985..."
28,TACNA,POCOLLAY,"{'rings': [[[-69.93216798026805, -17.926121110..."
29,TACNA,CALANA,"{'rings': [[[-70.11434372987918, -17.909730485..."
30,TACNA,TACNA,"{'rings': [[[-70.27632810505798, -17.984019485..."
31,TACNA,SAMA,"{'rings': [[[-70.57801760592776, -17.812158235..."
32,TACNA,CIUDAD NUEVA,"{'rings': [[[-70.07199810542818, -17.718091734..."
35,TACNA,ALTO DE LA ALIANZA,"{'rings': [[[-70.15002735467863, -17.599273485..."
36,TACNA,PACHIA,"{'rings': [[[-69.93100585383706, -17.590230485..."
40,TACNA,INCLAN,"{'rings': [[[-70.37032122984084, -17.460498235..."
43,TACNA,PALCA,"{'rings': [[[-69.58810997889236, -17.401889984..."


In [ ]:
positive_cases["DISTRITO"].loc[positive_cases["DISTRITO"]  == "CORONEL GREGORIO ALBARRACIN L."]="CORONEL GREGORIO ALBARRACIN LANCHIPA"

In [129]:
positive_cases.loc[positive_cases["DISTRITO"]  == "CORONEL GREGORIO ALBARRACIN LANCHIPA"]

,UUID,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO
105323,808e0af728ac600aa2c66c840e57942c,TACNA,TACNA,CORONEL GREGORIO ALBARRACIN LANCHIPA,PCR,28.0,MASCULINO,4/04/2020
105328,beabf18a22b5613cf7585447b185d873,TACNA,TACNA,CORONEL GREGORIO ALBARRACIN LANCHIPA,PCR,36.0,FEMENINO,25/04/2020
105342,3ad0f39e3ce82be9a42924b45c459e10,TACNA,TACNA,CORONEL GREGORIO ALBARRACIN LANCHIPA,PR,34.0,FEMENINO,19/05/2020
105348,6be53318b71abe3a5e4db5360e8a3dd1,TACNA,TACNA,CORONEL GREGORIO ALBARRACIN LANCHIPA,PR,48.0,FEMENINO,15/05/2020
105350,99c2a5714671df097ef2ca906287b709,TACNA,TACNA,CORONEL GREGORIO ALBARRACIN LANCHIPA,PR,66.0,FEMENINO,19/05/2020
105351,b17db124c7444f22c7f3e4737aca4067,TACNA,TACNA,CORONEL GREGORIO ALBARRACIN LANCHIPA,PR,36.0,FEMENINO,19/05/2020
105352,35d5e5d4531ce48a429fbe7f90074bf5,TACNA,TACNA,CORONEL GREGORIO ALBARRACIN LANCHIPA,PR,49.0,FEMENINO,15/05/2020
105363,3f053bbe8be62e11f966d24a3a8154e2,TACNA,TACNA,CORONEL GREGORIO ALBARRACIN LANCHIPA,PR,63.0,FEMENINO,15/05/2020
105370,77b346ee1fec18191ff513669ca6acab,TACNA,TACNA,CORONEL GREGORIO ALBARRACIN LANCHIPA,PR,52.0,FEMENINO,16/05/2020
105382,c52ff2795a1263fc7ac053559ae95f98,TACNA,TACNA,CORONEL GREGORIO ALBARRACIN LANCHIPA,PR,27.0,MASCULINO,5/05/2020


## Join the data

In [131]:
geo_df = pd.merge(positive_cases, district, how = "left", on = ["PROVINCIA", "DISTRITO"])

In [133]:
geo_df

,UUID,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO,SHAPE
0,7320cabdc1aaca6c59014cae76a134e6,AMAZONAS,BAGUA,BAGUA,PCR,15.0,MASCULINO,7/04/2020,"{'rings': [[[-78.53754373795311, -5.4864195991..."
1,e81602051997ace8340bb8c18fe24c65,AMAZONAS,BAGUA,COPALLIN,PCR,73.0,MASCULINO,7/04/2020,"{'rings': [[[-78.3240911117154, -5.56973172423..."
2,cecdbf10074dbc011ae05b3cbd320a6f,AMAZONAS,BONGARA,JAZAN,PCR,85.0,MASCULINO,9/04/2020,"{'rings': [[[-78.05100198725188, -5.9035078486..."
3,71ecb6bccb248b0bb2ac72ed51b5e979,AMAZONAS,CHACHAPOYAS,HUANCAS,PCR,31.0,MASCULINO,9/04/2020,"{'rings': [[[-77.87975586214327, -6.1023280986..."
4,566af4276cbe9359abe93f9aa86396c3,AMAZONAS,BAGUA,COPALLIN,PCR,41.0,MASCULINO,9/04/2020,"{'rings': [[[-78.3240911117154, -5.56973172423..."
...,...,...,...,...,...,...,...,...,...
108764,099e9eeeb47ec70204a5e55c9f5306cb,UCAYALI,CORONEL PORTILLO,CALLERIA,PR,22.0,MASCULINO,22/04/2020,"{'rings': [[[-74.4714456089277, -7.27617472568..."
108765,3106814086998cee9282e68aaf4b8d08,UCAYALI,CORONEL PORTILLO,CALLERIA,PR,40.0,MASCULINO,2/05/2020,"{'rings': [[[-74.4714456089277, -7.27617472568..."
108766,63b74ab867a3ff1ff2f6068af0fb70a2,UCAYALI,CORONEL PORTILLO,CALLERIA,PR,23.0,MASCULINO,30/04/2020,"{'rings': [[[-74.4714456089277, -7.27617472568..."
108767,b964a37db73e3f1fe350b4acf7ce29fc,UCAYALI,CORONEL PORTILLO,YARINACOCHA,PR,64.0,MASCULINO,9/05/2020,"{'rings': [[[-74.60663473410568, -8.0537592264..."


## Export Table to Feature Class

In [135]:
#Exporting quantity of positive cases Covid 19 
cifras =pd.DataFrame(geo_df.groupby(['PROVINCIA', 'DISTRITO']).count()['UUID'])
cifras = pd.merge(cifras, district, how = "left", on = ["PROVINCIA", "DISTRITO"])
cifras.spatial.to_featureclass(os.path.join(outworkspace,"CIFRAS_COVID19"))

'C:\\Users\\jf_ph\\OneDrive\\Documentos\\ArcGIS\\Projects\\Covid19\\Outcomes.gdb\\CIFRAS_COVID19'

In [ ]:
data = geo_df.spatial.to_featureclass(os.path.join(outworkspace,"DATA_COVID19"))